# MBTA Network Data (Geospatial)

This tutorial will guide you through geospatial (GIS) datasets for MBTA's transit networks (rapid transit, buses, commuter rail).

_For an introduction of geospatial data, see [this QGIS tutorial](geospatial_qgis_example.ipynb). For more detailed network data that includes full schedules and frequency of transit lines, see [this GTFS tutorial](mbta_gtfs_data.ipynb)._

## Visual Exploration

### MBTA's Static System Maps

As an initial overview of the system, the MBTA provides [static PDF maps](https://www.mbta.com/maps), including the schematic subway map, the bus network, and commuter rail fare zones.

For example, here's the map of Downtown Boston and nearby neighborhoods:

![Downtown Map with rapid transit, buses and the street grid. This map includes portions of Boston, Brookline, Cambridge, Charlestown, and Somerville.](https://cdn.mbta.com/sites/default/files/styles/max_2600x2600/public/media/2025-08/2025-08-25-downtown-map.jpg?itok=O2yDcTxH)

### MassMapper

[MassMapper](https://maps.massgis.digital.mass.gov/MassMapper/MassMapper.html?bl=MassGIS%20Basemap__100&l=massgis%3AGISDATA.MBTA_NODE__GISDATA.MBTA_NODE%3A%3ADefault__ON__100%2Cmassgis%3AGISDATA.MBTA_ARC__GISDATA.MBTA_ARC%3A%3ADefault__ON__100%2Cmassgis%3AGISDATA.MBTA_NODE__GISDATA.MBTA_NODE%3A%3ALabels__ON__100%2CBasemaps_L3Parcels____ON__100&b=-71.17148843849154%2C42.36958967917281%2C-71.04883638465853%2C42.41649765155441) is an interactive tool by the Massachusetts state government that displays geospatial data from the official GIS database. The link is loaded with all rapid transit lines and stations; you can add other layers such as the US Census data, satellite imagery, and other property and infrastructure maps.

[Here is the Mass.gov information page](https://www.mass.gov/info-details/massgis-data-mbta-rapid-transit) that describes the data, which you can also download as GIS datasets.

![MassMapper with MBTA rapid transit lines and stations.](images/massmapper_rapid_transit.png)

### MBTA Bus Route Profiles

For information on individual bus routes, a good source of visual data is [MBTA Bus Route Profiles](https://www.mbta.com/projects/better-bus-project/update/bus-route-profiles-now-available). Written in 2018, these are detailed reports of each bus route with descriptions and analysis, including scheduled frequencies, ridership, crowding, on-time performance etc. 
Note that the data is pre-Covid.

Details have been elaborated in our [MBTA ridership data tutorial](mbta_ridership_data.ipynb).

## Provided Dataset: MBTA Network

We have provided [``data/mbta_network.gpkg``](data/mbta_network.gpkg) in this GitHub repo, which contains GeoPackage layers of routes and stops on the MBTA rapid transit, buses and commuter rail networks. It retrieves external datasets from MassGIS Data Hub linked below. A more detailed documentation is available in [``data/README.md``](data/). 

The rest of this tutorial contains:

* Links to original data sources with documentations
* What the additional ``_gtfs`` layers in the provided dataset mean
* Where and how to download additional data layers (online or in Python)

## Background



### Data Portals

You can download GIS data on MBTA network from **MassGIS**, the state's official database. It can be accessed through either of the two web portals:

1. [**Via the MassGIS Data Hub.**](https://gis.data.mass.gov/) This is an ArcGIS server that includes datasets from many agencies, via the search function or browsing through categories. GIS datasets can be viewed online, downloaded in code via APIs (demonstrated below), or downloaded via the [geodatadownloader](https://geodatadownloader.com/) web tool.

2. [**Via the Mass.gov website.**](https://www.mass.gov/info-details/massgis-data-layers) The webpage gives a static list of data layers, each of which can be downloaded easily as ZIP files. It also typically provides better documentation.

Both sources contain additional data categories beyond transit, such as demographics, other infrastructure, and environmental monitoring. Most data layers can be viewed in MassMapper.

Additional data portals include the [MassDOT GeoDOT Open Data Portal](https://geodot-massdot.hub.arcgis.com/pages/open-data-portal) and the [MBTA Open Data Portal](https://mbta-massdot.opendata.arcgis.com/).

### MBTA Network Datasets

Geometries for each of MBTA's transit mode can be found as follows; both sources are equivalent. All of them are included in [``data/mbta_network.gpkg``](data/mbta_network.gpkg). For documentation, use the Mass.gov link.

* **Rapid transit:** [via MassGIS Data Hub](https://gis.data.mass.gov/maps/a9e4d01cbfae407fbf5afe67c5382fde/explore), or [via Mass.gov](https://www.mass.gov/info-details/massgis-data-mbta-rapid-transit)
* **Commuter rail:** [via MassGIS Data Hub](https://gis.data.mass.gov/maps/b741e3542a9f40898d49e776452efe63), or [via Mass.gov](https://www.mass.gov/info-details/massgis-data-trains)
* **Buses:** [via MassGIS Data Hub](https://gis.data.mass.gov/maps/cef8d0fe8b9d49fe9aa8f1b7524c6ac2/explore), or [via Mass.gov](https://www.mass.gov/info-details/massgis-data-mbta-bus-routes-and-stops)

### Additional Layers Parsed from GTFS Schedules

Alternatively, all three modes are in the [GTFS_Systemwide layers (via MassDOT ArcGIS Server)](https://gis.massdot.state.ma.us/arcgis/rest/services/Multimodal/GTFS_Systemwide/MapServer), included in [``data/mbta_network.gpkg``](data/mbta_network.gpkg) as ``_gtfs`` layers. These datasets are parsed directly from the latest GTFS schedule bundles, with much more information for stops (e.g. accessibility and sidewalk conditions). They do not list which line the station is on, but you can join them with [data/MBTA_Rapid_Transit_Stop_Orders.csv](data/MBTA_Rapid_Transit_Stop_Orders.csv) for rapid transit.

_For a quick introduction of GTFS, see [this other tutorial](mbta_gtfs_data.ipynb)._

## Importing the MBTA Network in Python

As an example, we download the [MBTA Rapid Transit](https://gis.data.mass.gov/maps/a9e4d01cbfae407fbf5afe67c5382fde/explore) routes data from MassGIS Data Hub ArcGIS Server. This uses our helper function ``query_arcgis_feature_server`` in [``hackathon_utils.py``](hackathon_utils.py). 

All URLs below are API links. To obtain the API link from MassGIS Data Hub (or any other ArcGIS Server), from the map view of an ArcGIS dataset online, follow **"I want to use this" – "View Data Source"**.

In [1]:
MBTA_NETWORK_URLS = {
    # URLs from MassGIS Data Hub Server; note that the last number is Layer ID
    "rapid_transit_routes": "https://arcgisserver.digital.mass.gov/arcgisserver/rest/services/AGOL/MBTA_Rapid_Transit/MapServer/4",
    "rapid_transit_stops": "https://arcgisserver.digital.mass.gov/arcgisserver/rest/services/AGOL/MBTA_Rapid_Transit/MapServer/0",
    "commuter_rail_routes": "https://arcgisserver.digital.mass.gov/arcgisserver/rest/services/AGOL/MBTA_Commuter_Rail/MapServer/3",
    "commuter_rail_stops": "https://arcgisserver.digital.mass.gov/arcgisserver/rest/services/AGOL/MBTA_Commuter_Rail/MapServer/0",
    # "bus_routes": "https://arcgisserver.digital.mass.gov/arcgisserver/rest/services/AGOL/MBTA_Bus/MapServer/3",
    # ^ Can't be downloaded automatically due to bugs
    "bus_stops": "https://arcgisserver.digital.mass.gov/arcgisserver/rest/services/AGOL/MBTA_Bus/MapServer/0",

    # Below are datasets parsed from the GTFS_Systemwide layers (more information but less trivial connections between stops and routes)
    "rapid_transit_routes_gtfs": "https://gis.massdot.state.ma.us/arcgis/rest/services/Multimodal/GTFS_Systemwide/MapServer/1",
    "rapid_transit_stops_gtfs": "https://gis.massdot.state.ma.us/arcgis/rest/services/Multimodal/GTFS_Systemwide/MapServer/0",
    # "commuter_rail_routes_gtfs": "https://gis.massdot.state.ma.us/arcgis/rest/services/Multimodal/GTFS_Systemwide/MapServer/3",
    # ^ Data source is bugged (due to specifying the wrong filter on the MassDOT end)
    "commuter_rail_stops_gtfs": "https://gis.massdot.state.ma.us/arcgis/rest/services/Multimodal/GTFS_Systemwide/MapServer/2",
    "bus_routes_gtfs": "https://gis.massdot.state.ma.us/arcgis/rest/services/Multimodal/GTFS_Systemwide/MapServer/5",
    "bus_stops_gtfs": "https://gis.massdot.state.ma.us/arcgis/rest/services/Multimodal/GTFS_Systemwide/MapServer/4",
}

In [2]:
from hackathon_utils import query_arcgis_feature_server
import geopandas as gpd

QUERY_DATASET = "rapid_transit_routes"

df = query_arcgis_feature_server(MBTA_NETWORK_URLS[QUERY_DATASET])
# df is now a GeoDataFrame in geopandas

In [3]:
df

,OBJECTID,LINE,ROUTE,GRADE,SHAPE.LEN,geometry
0,1,ORANGE,Forest Hills to Oak Grove,1,1342.326419,"LINESTRING (-7912281.363 5220867.006, -7912281..."
1,2,ORANGE,Forest Hills to Oak Grove,4,368.215569,"LINESTRING (-7912282.784 5220367.321, -7912280..."
2,3,ORANGE,Forest Hills to Oak Grove,1,3010.706909,"LINESTRING (-7911045.491 5216800.392, -7911100..."
3,4,GREEN,C - Cleveland Circle,2,3803.501983,"LINESTRING (-7920302.427 5211466.148, -7919912..."
4,5,GREEN,D E,4,1621.644873,"LINESTRING (-7910908.561 5215932.593, -7910965..."
...,...,...,...,...,...,...
136,137,SILVER,SL2,3,10.652447,"LINESTRING (-7907268.612 5212757.699, -7907263..."
137,138,GREEN,D - Union Square,4,906.641859,"LINESTRING (-7912688.035 5217184.153, -7912732..."
138,139,GREEN,E - Medford/Tufts,1,4710.683462,"MULTILINESTRING ((-7913060.228 5217614.242, -7..."
139,140,GREEN,E - Medford/Tufts,4,422.992560,"LINESTRING (-7912688.035 5217184.153, -7912688..."


You can save the data locally so that it can be opened in QGIS or ArcGIS. 

The code below writes in GeoPackage format, which allows each ``.gpkg`` file to contain several layers. We use ``QUERY_DATASET`` as the layer name.

In [4]:
import os

OUTPUT_FILEPATH = "data_downloads/network.gpkg"
output_dir = os.path.dirname(OUTPUT_FILEPATH)
os.makedirs(output_dir, exist_ok=True)

df.to_file(OUTPUT_FILEPATH, driver='GPKG', layer=QUERY_DATASET, mode='w')